In [2]:
# coding:utf-8  
###  ====================== 後端測試區 - TF-IDF =========================
import json
import jieba
import re
import os  
import sys 
import time
from datetime import datetime
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import TfidfVectorizer




### 載入要parse的資料
allData = []
for dataName in range(1,2):    
    with open('news.json'.format(dataName), 'r',encoding='utf-8') as f:
        data = json.load(f)
        print('Deal with data{}.json'.format(dataName))
        allData += data
### 載入繁體擴充字典
jieba.set_dictionary('dict.txt-v2.big')    

### 載入停用字
print('Input Stop Words')
stopwordset = set()
with open('stopwords.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

print('Start Jieba Cut')
totalContent = []
### 從data中挑1000篇文章切詞 - 特別注意，這行的迴圈數字決定參予tf-idf計算的文章數量
for i in range(0,50):
    article = ''
    content = allData[i]['Content']
    
    ### 斷詞, 產生一個結巴物件
    words = jieba.cut(content, cut_all=False)
    for word in words:   
        ### 正規表達式，只針對文字處理
        m = re.match(r'^[\u4E00-\u9FFFa-zA-Z]+$',word )
        if m is not None:
            if word not in stopwordset:
                article+=word
                article+=' '
    totalContent.append(article)

## 用來裝全部文章的權重值
textWeightList = []

## 用來裝前5權重值
tagList = []




### TF-IDF 開始計算
print('Start Tf-Idf')
if __name__ == "__main__":  
   
    tfIdfVectorizer = TfidfVectorizer()    
    tfIdf = tfIdfVectorizer.fit_transform(totalContent)    
    ## 取得詞袋模型中的所有詞語 
    myWord=tfIdfVectorizer.get_feature_names()        
    ## 將tf-idf矩陣抽取出來， j詞在第i篇文章中的tf-idf權重 
    weight=tfIdf.toarray()        
    for i in range(len(weight)):
        ## 用來裝單篇權重值
        textMining = {}       
        for j in range(len(myWord)):             
            ##print( myWord[j],weight[i][j] ) 
            textMining[myWord[j]] = weight[i][j]
        textWeightList.append(textMining)


print('Sorted the Weight of KeyWords')
### 權重排序 lanbda式
for oneArticle in textWeightList:
    dict= sorted(oneArticle.items(), key=lambda d:d[1], reverse = True)

    ### 決定要放幾個tag
    tag = []
    for i in range(0,5):
        tag.append(dict[i][0])
    tagList.append(tag)
        
### 載入要parse的資料
for i in range(len(tagList)):
    allData[i]['Tag'] =tagList[i]



## 讀取檔案
listName = []

with open('nameList.txt','r',encoding='utf-8') as nl:
    for line in nl:
        name = line.split('\n')[0]
        listName.append(name.lower())

## 設定正規表達式
reRule = '[a-zA-Z{" "}{\.}{\,}{\-}a-zA-Z]+'
for i in range(0,len(tagList)):
    tagByName = []
    oneNew = allData[i]['Content']
    ## 透過正規表達留下英文詞彙
    reResult=re.findall(reRule, oneNew)
    for j in reResult:
        ## 比對名字是否為mlb球員
        if j.lower() in listName:
            print(j)
            tagByName.append(j.lower())
    allData[i]['TagByName'] = tagByName

Building prefix dict from C:\Users\ronin\Documents\github\dict.txt-v2.big ...


Deal with data1.json
Input Stop Words
Start Jieba Cut


Dumping model to file cache C:\Users\ronin\AppData\Local\Temp\jieba.ua037b1d9f9e0ba5d3e9261c38971b30a.cache
Loading model cost 1.468 seconds.
Prefix dict has been built succesfully.


Start Tf-Idf
Sorted the Weight of KeyWords
Chipper Jones
Julio Teheran
Greg Maddux
Tom Glavine
John Smoltz
Roger Clemens
Max Scherzer
Ben Zobrist
Miguel Cabrera
Matt Harvey
Ian Desmond
Bobby Parnell
Ryan Zimmerman
Jayson Werth
Ian Desmond
Kurt Suzuki
CC Sabathia
Joe Girardi
Kevin Gausman
Brett Gardner
Robinson Cano
Manny Machado
Adam Jones
Nate McLouth
T.J. McFarland
Buck Showalter
Jim Johnson
Tommy Hunter
Albert Pujols
J.B. Shuck
Mike Trout
Jered Weaver
Doug Fister
Melky Cabrera
Jose Bautista
Anthony Gose
Kevin Pillar
Rajai Davis
Emilio Bonifacio
John Gibbons
Chipper Jones
Derek Jeter
A-Rod
Alex Rodriguez
A-Rod
A-Rod
A-Rod
Joe Girardi
A-Rod
Josh Johnson
Allen Webster
Stephen Drew
Mike Napoli
Jose Bautista
David Ortiz
Edwin Encarnacion
Brett Cecil
Darren Oliver
Kevin McHale
Chris Davis
Albert Pujols
Brady Anderson
Robinson Cano
Jordan Zimmermann
Adam Wainwright
Mike Matheny
Matt Adams
Yadier Molina
Francisco Liriano
Pedro Alvarez
Garrett Jones
Jason Grilli
Doug Fister
John Gibbons
Nick

In [2]:
###  ====================== 後端測試區 - 建立陣列裝要上傳的資料  =========================

### 資料寫進要上傳的陣列

updateList = []


for j in range(len(tagList)):
    updateList.append(allData[j])

print('Start Change the data format')    
### 時間格式改為 YYYYMMDD
for i in range(0,len(updateList)): 
    if allData[i]['Link'].startswith('http://www.nownews'):
        newTime = updateList[i]['Date'].split('T')[0].split('-')[0]+updateList[i]['Date'].split('T')[0].split('-')[1]+updateList[i]['Date'].split('T')[0].split('-')[2]
    if allData[i]['Link'].startswith('https://www.sportsv'):
        newTime = updateList[i]['Date'].split('/')[0]+updateList[i]['Date'].split('/')[1]+updateList[i]['Date'].split('/')[2]
    if allData[i]['Link'].startswith('http://sports.ettoday'):
        newTime = updateList[i]['Date'].split(' ')[0].split('-')[0]+updateList[i]['Date'].split(' ')[0].split('-')[1]+updateList[i]['Date'].split(' ')[0].split('-')[2]
    allData[i]['YearDate'] = newTime

    


Start Change the data format


In [4]:
###  ====================== 後端測試區 - 建立連線上傳資料  =========================

### 將上傳陣列的東西寫進資料庫
### 全部是一次傳
### 2017的版本是要一筆一筆傳

import pymongo
print('start Insert Data to MongoDB')

client = pymongo.MongoClient("localhost", 27017)
db = client['spadeAce']
collection = db.allNews
for i in updateList:
    if collection.find_one({'Link':i['Link']}).count() < 1:
        collection.insert(i)
        print(i['Title'])
        print('data assess')
    else:
        print('data duplicated')

client.close()

start Insert Data to MongoDB


C:\Users\BIG DATA\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


MLB／2010年十大巨投排名 林瑟康、沙巴西亞高居一、二
data assess
MLB／保國聯西區王座！ 道奇3轟踢館洛磯成功 
data assess
MLB／球帽球衣不夠看 道奇新商品是棺材和骨灰罈
data assess
MLB／阿布瑞尤轉戰天使變選球導師 甚至有機會爭MVP
data assess
MLB／和鈴木一朗不是朋友？ 松井秀喜拒絕轉戰水手
data assess
MLB／血本無歸！大都會傷兵團薪資嚇人 總值29億新台幣
data assess
MLB／郭泓志球速狂飆156公里 投1局連3次三振 摘中繼點
data assess
MLB／柏奈特換捕手單場送12K 隊友不幫忙洋基仍吞敗
data assess
MLB／求穩定5號先發 洋基打算網羅紅襪棄將潘尼
data assess
MLB／潘尼將與波士頓Say Goodbye 確定放入紅襪釋出名單
data assess
MLB／小熊球迷個個毒舌 布萊德利：我超討厭主場！
data assess
MLB／威克菲爾重出江湖 7局好投助紅襪踩白襪
data assess
MLB／狂敲20轟！ 坎諾蛻變為洋基重砲手
data assess
MLB／休太久手臂生鏽？ 張伯倫4天後先發遇白襪 
data assess
MLB／客場表現比主場棒 史威許：我是洋基客場勇者！
data assess
MLB／洋基豪投猛打 9比2成功反咬遊騎兵
data assess
MLB／傷兵13人居大聯盟首位 大都會蹧蹋球員？
data assess
MLB／倪福德背粉紅色「菜鳥包」替大家服務 非常盡責
data assess
MLB／國聯外卡對決 勞恩德3分砲助巨人征服洛磯山
data assess
MLB／史威許短打失敗 吉拉迪錯誤戰術害洋基輸球？
data assess
MLB／小腿傷未完全恢復 鈴木一朗今仍未出賽
data assess
MLB／鐵捕確定明不出賽 波沙達柏奈特不和浮上檯面？
data assess
MLB／徹底栽在田澤純一手裡 洋基1比14慘遭紅襪踐踏
data assess
MLB／選秀狀元簽約過程超緊張 小史：好像在看慢動作！
data assess
MLB／兄弟象擊敗大聯盟紅人？ 海盜紀念球衣如出一轍
data assess
MLB／松井秀喜雙響包辦7打點 洋基20：11轟破紅襪
data as